In [16]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments, Trainer
import torch
from datasets import load_dataset
from peft import LoraConfig, get_peft_model
import pandas as pd


In [ ]:
# data = pd.read_csv('data/scoutieDataset_russian_jokes_with_vectors.csv')

dataset = load_dataset("csv", data_files="data/scoutieDataset_russian_jokes_with_vectors.csv")["train"] 

In [26]:
data = data[['text']]

In [27]:
data.to_csv('scoutieDataset_russian_jokes_with_vectors.csv')

In [10]:


tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-1.5B-Instruct", trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

def tokenize_function(examples):
    tokenized = tokenizer(examples["text"], truncation=True, max_length=128, padding="max_length")
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized


tokenized_dataset = dataset.map(tokenize_function, batched=True)

In [11]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)

cuda


In [12]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

model = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen2.5-1.5B-Instruct",
    quantization_config=bnb_config,
    trust_remote_code=True,
    device_map="auto"
)



In [13]:
peft_config = LoraConfig(
    task_type="CAUSAL_LM",  # Для генерации текста
    inference_mode=False,   # Режим обучения
    r=16,                   # Rank матриц LoRA (можно 4-16)
    lora_alpha=16,         # Коэффициент масштабирования
    lora_dropout=0,     # Дропаут для стабильности
    target_modules=[
        "q_proj",          # Проекции запросов
        "k_proj",          # Проекции ключей
        "v_proj",          # Проекции значений
        "o_proj",          # Выходные проекции
        "gate_proj",       # Врата в FFN-слоях
        "up_proj",         # Верхние проекции в FFN
        "down_proj",       # Нижние проекции в FFN
    ],
    bias="none",           # Не трогаем bias-параметры
)

model.add_adapter(adapter_config=peft_config, adapter_name='lora_2')
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

/home/t3fio/miniconda3/envs/jokes_dasha/lib/python3.10/site-packages/peft/mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/home/t3fio/miniconda3/envs/jokes_dasha/lib/python3.10/site-packages/peft/tuners/tuners_utils.py:167: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


trainable params: 18,464,768 || all params: 1,580,643,840 || trainable%: 1.1682


In [14]:
training_args = TrainingArguments(
    output_dir="./qwen2.5-lora-jokes/v2.0",
    per_device_train_batch_size=1,    # ⚠️ Маленький batch для 4-bit!
    gradient_accumulation_steps=4,    # "Виртуальный" batch_size = 2 * 4 = 8
    num_train_epochs=1,
    learning_rate=2e-4,               # Для LoRA можно повыше (1e-4 — 3e-4)
    optim="paged_adamw_8bit",         # Оптимизатор для 8-bit/4-bit
    logging_steps=10,
    save_steps=500,
    label_names=["labels"],
    fp16=True,                        # Если GPU не поддерживает bfloat16
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
)
trainer.train()

Step,Training Loss


KeyboardInterrupt: 